In [2]:
from dotenv import load_dotenv
import os

# Load document and start session 

In [3]:
load_dotenv()
H2O_API_KEY = os.getenv("H2O_API_KEY") 

In [4]:
from h2ogpte import H2OGPTE

client = H2OGPTE(
    address='https://h2ogpte.genai.h2o.ai',
    api_key= H2O_API_KEY,
)

# Create a new collection
collection_id = client.create_collection(
    name='First meeting transcirpt',
    description='Information related to the first group meeting',
)

In [5]:
with open('meeting_mintues.txt', 'rb') as f:
    meeting_data1 = client.upload('meeting_mintues.txt', f)
client.ingest_uploads(collection_id, [meeting_data1,])

Job(id='0761fd06-b19c-42ff-b778-9fbebdf79892', name='', passed=1.0, failed=0.0, progress=1.0, completed=True, canceled=False, date=datetime.datetime(2024, 3, 22, 8, 19, 1, tzinfo=TzInfo(UTC)), kind=<JobKind.IngestUploadsJob: 'IngestUploadsJob'>, statuses=[JobStatus(id='625aa094262e4c0e9c99b12a3e75ada6', status='Collecting done.'), JobStatus(id='16437e3aaf5f42d598304be264ce43f2', status='Collecting done.'), JobStatus(id='e846ed9e50e544479bc13d750ee29b36', status='Indexing done.')], errors=[], last_update_date=datetime.datetime(2024, 3, 22, 8, 19, 13, tzinfo=TzInfo(UTC)), duration='12s')

In [6]:
chat_session_id = client.create_chat_session(collection_id)

In [7]:
llm_list = client.get_llms() # default use first elem, index[0]
for llm in llm_list:
    print(llm)

{'base_model': 'mistralai/Mixtral-8x7B-Instruct-v0.1', 'prompt_type': 'mixtral', 'prompt_dict': {'promptA': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'promptB': '<<SYS>>\nYou are an AI that follows instructions extremely well and as helpful as possible.\n<</SYS>>\n\n', 'PreInstruct': '<s> [INST] ', 'PreInput': None, 'PreResponse': '[/INST]', 'terminate_response': ['[INST]', '</s>'], 'chat_sep': ' ', 'chat_turn_sep': '</s> ', 'humanstr': '[INST]', 'botstr': '[/INST]', 'generates_leading_space': False, 'system_prompt': 'You are an AI that follows instructions extremely well and as helpful as possible.', 'can_handle_system_prompt': True}, 'load_8bit': False, 'load_4bit': False, 'low_bit_mode': 1, 'load_half': False, 'use_flash_attention_2': False, 'load_gptq': '', 'load_awq': '', 'load_exllama': False, 'use_safetensors': False, 'revision': None, 'use_gpu_id': False, 'gpu_id': None, 'compile_model': None, 'use_cache': None,

In [8]:
def rag_chat(client, chat_session_id, main_prompt, system_prompt):
    with client.connect(chat_session_id) as session:
        reply = session.query(
            message=main_prompt,
            system_prompt=system_prompt,
            rag_config={
            "rag_type": "rag", # https://h2oai.github.io/h2ogpte/getting_started.html#advanced-controls-for-document-q-a
            },
            llm="h2oai/h2ogpt-4096-llama2-70b-chat",
        )
        return reply

# Summarizer and system prompt 

In [9]:
system_prompt = """
You are a Meeting Assistant for a software company. You will be given the meeting log in the form of audio, powerpoint 
or PDF. Your general duties include but are not limited to summarising important points during the meeting, taking note 
of everyone's deliverables and organising all information in a clear and concise form. Upon reading your output all
members should have a clear understanding of the meeting objectives, agenda, and outcomes.
"""

In [26]:
summarizer_prompt = """
Meeting Assistant Prompt Template:

Please retrieve the following details based on the meeting input:

1. Attendees: Who attended this meeting? Please list the names and email addresses of participants.

2. Meeting Details:
    Date and Time: When did the meeting take place?
    Duration: How long did the meeting last?

3. Meeting Purpose: 
    Objective: What is the primary objective or purpose of the meeting?
    Desired Outcome: What outcomes do you hope to achieve by the end of the meeting?

4. Meeting summary: Provide a concise summary of the entire meeting by listing out all the main highlights
                    such that participants not in the meeting will be able to understand what happened during the meeting.
    
5. Action Items:
    Deliverables: What is the distribution of workload discussed during the meeting?
    Assign: For each attendee list out in detail all the work assigned to them, be as precise and detailed as possible.

Present points 1-5 in the following format: 
{"Attendees": "", "Meeting Details": ["Date and Time":"", "Duration":""], 
"Meeting Details": ["Objective":"", "Desired Outcome":""], "Meeting summary":"", 
"Action Items": ["Deliverables":"", "Assign":""]
} 
"""

In [27]:
meeting_summary_response = rag_chat(client, chat_session_id, summarizer_prompt, system_prompt)
print(meeting_summary_response.content)

Attendees: Ryan_Edward, Jeremy_PH, Hannah Nga, Chan Yi Ru Micole, Ben_CH, Eugene_YJ

Meeting Details:
Date and Time: 22:17:11 - 22:55:14
Duration: 38 minutes

Meeting Purpose:
Objective: To discuss the development of a chatbot that can summarize meetings
Desired Outcome: To have a clear understanding of the meeting objectives, agenda, and outcomes

Meeting Summary:

* Discussed the possibility of creating a chatbot that can summarize meetings
* Considered the idea of using a bare bones model and pushing it out by the next meeting
* Talked about the importance of having a clear understanding of the meeting objectives, agenda, and outcomes
* Mentioned the possibility of extending the chatbot's functionality to include sentiment analysis
* Decided to have a follow-up meeting to discuss the progress of the chatbot development

Action Items:

Deliverables:

* Develop a chatbot that can summarize meetings
* Create a user interface for the chatbot
* Test the chatbot and make any necessary adj